In [26]:
import sys
sys.path.append('../RecSysRep/')

In [27]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, _, _, _, _ = ld.getCOOs()
ICM_all = ld.getICMall()

In [28]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=1999)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions


In [29]:
import os

ofp = "../models_temp/Similarity_Hybrid/"

models_to_combine_best = {
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' : {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_7': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'icm_3': {'shrink': 2211, 'topK': 188, 'feature_weighting': 'TF-IDF', 'normalize': True},
                            'icm_9': {'shrink': 3568, 'topK': 986, 'feature_weighting': 'TF-IDF', 'normalize': True},
                            'icm_all': {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'UserKNN' : {'topK': 448, 'similarity': 'cosine', 'shrink': 756, 'normalize': True, 'feature_weighting': 'TF-IDF', 'URM_bias': True},
                         }

In [30]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [31]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid

In [32]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.HybridScores.DifferentStructure import TwoDifferentModelRecommender

In [33]:
# recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender2 = ItemKNNCBFRecommender(URM_train, ICM_all)

# model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
model_init(recommender2, 'icm_all_1999', models_to_combine_best['icm_all'])

../models_temp/Similarity_Hybrid/icm_all_1999.zip
Model does not exists, creating...
Similarity column 18059 (100.0%), 2734.84 column/sec. Elapsed time 6.60 sec
ItemKNNCBFRecommender: Saving model in file '../models_temp/Similarity_Hybrid/icm_all_1999'
ItemKNNCBFRecommender: Saving complete


In [41]:
import numpy as np

n_users = URM_train.shape[0]

new_URM = recommender2._compute_item_score(np.arange(n_users))

In [42]:
print(new_URM)

[[0.00274971 0.00826034 0.0057823  ... 0.000691   0.01110387 0.00404164]
 [0.04856354 0.06859856 0.0293282  ... 0.24117275 0.13705681 0.06411437]
 [0.01050267 0.02719916 0.00473097 ... 0.01406862 0.03240957 0.02359655]
 ...
 [0.01027202 0.03058787 0.01385761 ... 0.0498495  0.04843123 0.03399358]
 [0.00600578 0.01708084 0.00891311 ... 0.02498849 0.03078481 0.04979611]
 [0.01223729 0.033321   0.04553975 ... 0.08713911 0.03447174 0.01713734]]


In [36]:
# mask_URM = new_URM
# mask_URM[mask_URM>0] = 1

# print(np.sum(mask_URM, axis=1))
# print(mask_URM.shape)

In [46]:
thresh_URM = new_URM

for user in np.arange(n_users):
    ranking_list, scores = recommender2.recommend(user, cutoff = 100, return_scores = True)
    threshold = scores[0][ranking_list[-1]]
    # print(threshold)
    thresh_URM[user][thresh_URM[user] < threshold] = 0
    
print(thresh_URM)
print(new_URM)

0.24626008
0.4046448
0.11179665
0.08919274
0.4867709
0.19854254
0.0683455
0.17835657
0.16233444
0.22449675
0.2638688
0.3278687
0.15036643
0.16519114
0.23502322
0.2719395
0.19812457
0.1323078
0.30567023
0.22937346
0.3289053
0.41923735
0.21840313
0.17774034
0.39271757
0.45640478
0.36668965
0.25555277
0.2207242
0.258543
0.1790744
0.43050274
0.10107691
0.2795502
0.19068226
0.58620393
0.32237718
0.25265476
0.42645636
0.23602581
0.1726434
0.12247052
0.46860507
0.16615027
0.26764867
0.3913868
0.25654113
0.085066386
0.4284243
0.241781
0.18750222
0.124972686
0.31920406
0.321211
0.0279943
0.019866748
0.3627228
0.05875686
0.06695733
0.25891364
0.09356944
0.32699695
0.38436908
0.22892325
0.1763616
0.37434852
0.18255293
0.2851916
0.32065493
0.23832966
0.37723386
0.23458047
0.44170824
0.27252212
0.29252005
0.5300876
0.16468428
0.56856376
0.56340396
0.048260827
0.2540594
0.054972943
0.14079762
0.23535016
0.2832302
0.0595242
0.6500987
0.12168275
0.12311815
0.3196254
0.26877922
0.14487772
0.57355666
0.

0.35915405
0.2865485
0.11604138
0.09575202
0.2641993
0.18229933
0.2364699
0.23608255
0.54065347
0.3221695
0.27893066
0.35241336
0.06928705
0.40312532
0.28679854
0.026193468
0.17126895
0.37536678
0.46678233
0.2236422
0.3156964
0.183206
0.4533467
0.33598846
0.31118053
0.50089717
0.26591584
0.50822556
0.15574981
0.19555672
0.2450616
0.10905493
0.35152018
0.19277652
0.13682038
0.24043563
0.07201056
0.09277473
0.2074441
0.5124226
0.4823731
0.24114892
0.036030415
0.40550753
0.118465126
0.30119225
0.22056565
0.09438153
0.26915804
0.21989965
0.25726786
0.43224743
0.32224643
0.20090878
0.47744936
0.0782287
0.4782878
0.39111885
0.110352315
0.2593522
0.33601153
0.2711906
0.3162522
0.44284454
0.2037428
0.51010025
0.3072821
0.42947304
0.33573693
0.3388808
0.34211332
0.54684097
0.33274776
0.052326985
0.47630984
0.3581127
0.2694186
0.30819365
0.12262911
0.2814835
0.24263628
0.49754775
0.07411677
0.04879171
0.23368482
0.49462545
0.41114867
0.80970436
0.36798656
0.23068929
0.2749759
0.23333262
0.219823

0.29132575
0.39922208
0.27225244
0.41680866
0.32828757
0.25204584
0.28239495
0.28412798
0.21619192
0.044193596
0.28216273
0.26846716
0.31527215
0.41568837
0.25233266
0.07822542
0.12126721
0.28647777
0.2699607
0.35729197
0.45241222
0.27783024
0.47497588
0.1758889
0.11982704
0.26991174
0.22999422
0.41698983
0.08548054
0.16401425
0.65404046
0.13489333
0.048581034
0.19832802
0.20337461
0.26947483
0.081759036
0.089879155
0.524855
0.4186243
0.2570916
0.4043472
0.15912288
0.29342952
0.51465446
0.08940045
0.017129764
0.2530978
0.1687776
0.24808979
0.21654755
0.18232873
0.28967825
0.46560973
0.116886266
0.18496536
0.33734757
0.26111296
0.4360125
0.41608816
0.22497846
0.22214971
0.22500023
0.5391162
0.37326854
0.30233437
0.32699195
0.626066
0.26021284
0.31294033
0.17260496
0.42110816
0.0088666035
0.63185495
0.16539745
0.30405805
0.54278046
0.21082993
0.101998076
0.25311947
0.25793862
0.14393768
0.34007218
0.24815153
0.23961541
0.15883057
0.15757033
0.2960641
0.26278985
0.76996005
0.45006695
0.36

0.24991645
0.20775193
0.3099579
0.33404076
0.21967484
0.20394416
0.121369794
0.042684276
0.35712826
0.3620271
0.21220739
0.40436372
0.15753315
0.1726994
0.28459102
0.30335644
0.14058308
0.0705861
0.36049056
0.24397989
0.3084996
0.23237252
0.36785814
0.03144231
0.5473124
0.7175037
0.36377075
0.63898224
0.1981416
0.14597379
0.28705907
0.2010498
0.16997038
0.4029399
0.47350213
0.30278027
0.067355685
0.31274104
0.2674427
0.1652805
0.3155942
0.054450534
0.3359734
0.51809347
0.011442627
0.6010793
0.35121065
0.22610484
0.26328057
0.05316935
0.11472921
0.23109525
0.08759247
0.3608293
0.11289248
0.29256848
0.2417223
0.3176843
0.2219414
0.3820502
0.5272875
0.2446987
0.37324607
0.077012226
0.26716226
0.22531413
0.3096654
0.1114907
0.31844306
0.17335144
0.317033
0.12255152
0.14859587
0.40190393
0.21109857
0.3631887
0.118029274
0.3111658
0.19560365
0.25636628
0.16933464
0.34326306
0.14733417
0.5292682
0.073154144
0.48193106
0.55883485
0.1898112
0.45589632
0.21058726
0.29006425
0.08262957
0.35992882

0.5769267
0.07846618
0.45776927
0.08969009
0.30400598
0.22905312
0.18097375
0.25577182
0.10192816
0.23870344
0.3337006
0.15661968
0.28346404
0.4329621
0.42384455
0.27359778
0.24621019
0.40591133
0.11822467
0.4268745
0.27845886
0.2664264
0.21157922
0.21950583
0.46273988
0.19270861
0.32788655
0.47370285
0.19949786
0.0632665
0.21678743
0.42960876
0.024256535
0.2142082
0.24463879
0.33389926
0.46281263
0.30524713
0.20714453
0.39823842
0.1926146
0.21838349
0.19495197
0.20217572
0.18083854
0.13404799
0.2637475
0.072469
0.14666328
0.23031084
0.27541906
0.39331046
0.10275264
0.31622738
0.3161977
0.3933446
0.3747532
0.30609715
0.027175704
0.4121691
0.14593305
0.66177964
0.24198776
0.4352514
0.24103826
0.3765095
0.31703156
0.5696168
0.15202928
0.3149385
0.501922
0.40769503
0.26322728
0.18439847
0.3039461
0.23159744
0.6184598
0.349335
0.23112197
0.3613972
0.24665491
0.35244486
0.4233168
0.46447906
0.20327543
0.22085269
0.31135905
0.16089788
0.33734167
0.08830622
0.4190321
0.22844449
0.20986359
0.2

...

0.27760813
0.079145215
0.18511291
0.1371698
0.26541093
0.29599893
0.22957925
0.34500083
0.01986205
0.45013872
0.160255
0.1354065
0.13954306
0.47798818
0.36361134
0.42434835
0.2733521
0.28360942
0.4869606
0.12308565
0.25245446
0.21731973
0.26029915
0.1801323
0.08847336
0.4965778
0.4000231
0.2744721
0.24679498
0.21155988
0.42818978
0.32306045
0.23948213
0.19245769
0.25186485
0.30935702
0.3606766
0.29142895
0.45312828
0.1501638
0.32560867
0.18735972
0.1508948
0.25652593
0.42018256
0.35065842
0.1492669
0.1140609
0.4047877
0.33025232
0.11378341
0.17976667
0.52268875
0.6256146
0.2541536
0.23857713
0.1175728
0.08918348
0.3139892
0.26448712
0.30241778
0.34233683
0.009288223
0.18501826
0.2781473
0.17277594
0.23900838
0.016745824
0.48018712
0.35770252
0.011936929
0.3285446
0.23730128
0.30296662
0.39072528
0.55120003
0.22052641
0.6754668
0.23018643
0.24592653
0.32483736
0.7225854
0.52401733
0.252307
0.18160011
0.030146057
0.3474842
0.012564698
0.22780472
0.46232095
0.5624112
0.35960236
0.24947092

0.30127716
0.30113396
0.06609598
0.17091395
0.15899661
0.3886511
0.3605968
0.06424894
0.25478286
0.21472958
0.4531846
0.14260828
0.15956727
0.300616
0.12122991
0.26198164
0.30765367
0.18839161
0.36061436
0.10935999
0.39025646
0.18724713
0.36561573
0.51748157
0.15804102
0.42224044
0.20744643
0.46322253
0.4560134
0.2893786
0.33672595
0.07826664
0.46479377
0.568016
0.3338138
0.25252122
0.27302745
0.58227193
0.122041486
0.15253438
0.21333799
0.23727381
0.37770924
0.46559593
0.27663404
0.15939213
0.20659378
0.044080485
0.09710732
0.20147967
0.14114638
0.24513794
0.26176357
0.251279
0.20758453
0.044880453
0.36554596
0.32798177
0.3300848
0.07657409
0.26273173
0.33822176
0.64473027
0.39600006
0.47762862
0.19393335
0.29572508
0.28155732
0.12862153
0.29222384
0.5655622
0.22571976
0.13830763
0.22302966
0.110362984
0.13963112
0.25551426
0.2210878
0.28609556
0.3624914
0.31017065
0.12779862
0.22211978
0.14227362
0.12858856
0.22175318
0.108769834
0.33448377
0.35476166
0.13899292
0.68711156
0.2787501


0.37223125
0.10373614
0.17268956
0.067340285
0.14582804
0.27119058
0.45425212
0.22592767
0.09920251
0.2942779
0.21154633
0.2548219
0.15189292
0.07162954
0.31222323
0.33963057
0.3572033
0.27021348
0.5331808
0.12655683
0.4827997
0.23992656
0.48378724
0.3045753
0.29993397
0.33371922
0.1516725
0.113577835
0.3245652
0.2290532
0.44899032
0.15099224
0.17702498
0.23767784
0.11545321
0.7832103
0.23583971
0.44876876
0.2518469
0.62223583
0.338349
0.49584088
0.18307538
0.29336625
0.2863878
0.22483146
0.20291612
0.108315915
0.33405882
0.12247872
0.18630037
0.14865808
0.09686941
0.6252891
0.43547416
0.09670222
0.28246677
0.19025005
0.37590718
0.27542722
0.42591664
0.37703863
0.24969812
0.21379086
0.13593426
0.051394857
0.245338
0.028263649
0.33027574
0.35099316
0.013857245
0.26183552
0.16202375
0.17702924
0.30410543
0.31089848
0.3619298
0.25746858
0.21157236
0.19705066
0.16207032
0.041756958
0.6939099
0.015215883
0.062353235
0.2262767
0.22641076
0.29082802
0.28879803
0.23112744
0.4693914
0.21911274


0.3997632
0.22541833
0.52751434
0.317733
0.42631635
0.3951655
0.25031292
0.43818843
0.41181153
0.44396818
0.19458011
0.31248233
0.52208453
0.12637304
0.37643126
0.09774598
0.05378928
0.28414372
0.2946421
0.18871059
0.230371
0.22282185
0.093043596
0.094295084
0.2809065
0.3946419
0.5410232
0.29895326
0.16738176
0.133953
0.27878934
0.044512622
0.30640513
0.59657806
0.21988931
0.14029385
0.1460124
0.3164237
0.30171564
0.33761522
0.38876608
0.5634339
0.15587384
0.17193833
0.12003645
0.11451608
0.2959998
0.32615417
0.27176136
0.047575068
0.25665572
0.22633411
0.2771474
0.29747522
0.16576548
0.2912262
0.6943401
0.22554138
0.10829603
0.39145797
0.380646
0.3887927
0.3338903
0.14585091
0.30041555
0.13753957
0.1862627
0.16129287
0.2694275
0.28053737
0.22202927
0.31794694
0.5025557
0.54668665
0.30176336
0.18545696
0.1183218
0.48240477
0.1903952
0.27514192
0.45102203
0.3840814
0.2886602
0.35687244
0.48582444
0.24859615
0.07291586
0.30387598
0.21217531
0.29154927
0.2115824
0.25625798
0.4818156
0.353

0.21537802
0.4116766
0.10211128
0.38984686
0.14790626
0.2169418
0.053751726
0.08958715
0.15931752
0.38428128
0.30912918
0.20013909
0.2071827
0.22447631
0.47570202
0.059869505
0.31328803
0.29845598
0.063695654
0.4262465
0.23460136
0.11922163
0.392669
0.46386313
0.23278292
0.25309977
0.20477019
0.23111306
0.2997589
0.28232375
0.31716216
0.1420101
0.27785632
0.55687934
0.42151967
0.10780773
0.111314155
0.4910668
0.2979059
0.25055194
0.5329139
0.48658255
0.46740282
0.3839065
0.10557184
0.50253814
0.28426453
0.15802303
0.3878987
0.11241538
0.28889492
0.24300258
0.19974229
0.11654299
0.020772709
0.26614428
0.08716357
0.2326454
0.25576028
0.4327042
0.296555
0.056738283
0.31048736
0.19345735
0.18482205
0.1388964
0.090671144
0.23621574
0.19429767
0.048054963
0.47550166
0.07265644
0.03347066
0.22485566
0.39271986
0.20105605
0.360206
0.35515666
0.18773435
0.4174213
0.29732177
0.09211159
0.2452364
0.27527112
0.26952085
0.33970433
0.046903774
0.23962434
0.023702431
0.25177938
0.04018513
0.4839364
0

In [47]:
new_URM_th = sps.csr_matrix(thresh_URM)

In [48]:
new_URM_th.data

array([0.2705933 , 0.2705933 , 0.25436926, ..., 0.25428432, 0.24703683,
       0.27161312], dtype=float32)

In [49]:
recommender3 = MultiThreadSLIM_SLIMElasticNetRecommender(new_URM_th)
recommender3.fit(**models_to_combine_best['SLIMER'])


  0%|                                                                                        | 0/18059 [00:00<?, ?it/s]

SLIMElasticNetRecommender: URM Detected 5851 (32.4%) items with no interactions.



 12%|████████▉                                                                    | 2088/18059 [04:16<36:45,  7.24it/s]


 23%|██████████████████                                                           | 4232/18059 [08:32<18:33, 12.42it/s]


 35%|██████████████████████████▉                                                  | 6312/18059 [12:58<25:39,  7.63it/s]


...

 59%|████████████████████████████████████████████▍                               | 10568/18059 [22:08<13:07,  9.51it/s]


 71%|█████████████████████████████████████████████████████▊                      | 12776/18059 [26:33<09:07,  9.65it/s]


 82%|██████████████████████████████████████████████████████████████▋             | 14888/18059 [30:29<04:04, 12.94it/s]


 94%|███████████████████████████████████████████████████████████████████████▌    | 17000/18059 [34:41<02:07,  8.31it/s]


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [36:52<00:00, 12.41it/s]

In [50]:
r_d, _ = evaluator_validation.evaluateRecommender(recommender3)
print(r_d.loc[10]['MAP'])


EvaluatorHoldout: Processed 13646 (100.0%) in 4.83 sec. Users per second: 2826
0.010709513689691048
